In [1]:
import pandahouse as ph
import pandas as pd

In [6]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [8]:
query = """
    SELECT
      test_grp,
      uniqExact(st_id) AS total_students,
      ROUND(AVG(revenue), 2) AS ARPU,     -- считаем APRU
      ROUND(SUM(revenue) / SUM(was_active), 2) AS ARPAU,
      ROUND(
        SUM(CASE WHEN revenue == 0 THEN 0 ELSE 1 END) /    -- делим instances ненулевой выручки на кол-во студентов
            COUNT(st_id), 2) AS CR,
      ROUND(
          SUM(CASE WHEN revenue == 0 THEN 0 ELSE 1 END) /   -- делим instances ненулевой выручки на кол-во активных студентов
            SUM(was_active), 2) AS active_CR,
      ROUND(
          SUM(CASE WHEN subject_bought == 'Math' THEN 1 ELSE 0 END) /   -- находим cr для математиков
            SUM(CASE WHEN did_math > 0 THEN 1 ELSE 0 END), 2) AS math_CR
    FROM(
        SELECT
          studs.st_id AS st_id,            -- достаем из checks инф-ю об оплатах и прикрепляем ее к инф-е об активных студентах
          studs.test_grp AS test_grp,
          final_project_check.money AS revenue,
          subject AS subject_bought  -- предмет из checks записываем как купленный
        FROM
          default.studs
          LEFT JOIN default.final_project_check ON default.studs.st_id = default.final_project_check.st_id
      ) AS a
      LEFT JOIN (
        SELECT
          r.active_id AS active_id,
          MAX(               -- так как джойн многоуровневый, быстрее будет писать conditional через window
            CASE WHEN isNotNull(active_id) THEN True else False end
          ) OVER (PARTITION BY st_id) as was_active
        FROM
          default.studs AS l
          INNER JOIN (                 -- выбираем из peas активных студентов
            SELECT
              st_id AS active_id
            FROM
              default.peas
            GROUP BY
              active_id
            HAVING
              SUM(correct) > 10
          ) AS r ON l.st_id = r.active_id
      ) AS b ON b.active_id = a.st_id
      LEFT JOIN (
        SELECT
          r.active_math_id AS active_math_id,
          MAX(                       -- так как джойн многоуровневый, быстрее будет писать conditional через window
            CASE WHEN isNotNull(active_math_id) THEN True else False end
          ) OVER (PARTITION BY st_id) as did_math  
        FROM
          default.studs AS l
          INNER JOIN (                -- выбираем из peas активных студентов по математике
            SELECT
              st_id AS active_math_id
            FROM
              default.peas
            GROUP BY
              active_math_id
            HAVING
              (subject == 'Math')
              AND (SUM(correct) >= 2)
          ) AS r ON l.st_id == r.active_math_id
      ) AS c ON a.st_id == c.active_math_id
    GROUP BY test_grp
"""
df = ph.read_clickhouse(query, connection=connection_default)
df

,test_grp,total_students,ARPU,ARPAU,CR,active_CR,math_CR
0,control,305,4496.75,10653.85,0.06,0.14,0.16
1,pilot,295,11094.77,32028.30,0.14,0.41,0.35
